In [375]:
import pandas as pd
import numpy as np
import math

In [376]:
def get_numpy_data(data_frame, features, output):
    selected_data_frame = data_frame[features]
    output_array = data_frame[output].to_numpy()
    np_selected_data_frame = selected_data_frame.to_numpy()
    total_row = np_selected_data_frame.shape[0]
    np_ones = np.ones(total_row, dtype=int).reshape(total_row, 1)
    
    features_array = np.append(np_ones, np_selected_data_frame, axis=1)
    
    return (features_array, output_array)

In [377]:
def predict_outcome(feature_matrix, weights):
    return np.dot(feature_matrix, weights)

In [378]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    return (features/norms, norms)

In [379]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_outcome(feature_matrix, weights)
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:, i] * (output - prediction + weights[i] * feature_matrix[:, i]))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

## Effect of L1 penalty

Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added).
- Use the entire ‘sales’ dataset for now.
- Normalize columns of the feature matrix. 
- Save the norms of original features as ‘norms’.
- Set initial weights to [1,4,1].
- Make predictions with feature matrix and initial weights.
- Compute values of ro[i], where

```python
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```


In [423]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

(feature_matrix, output) = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')

In [424]:
initial_weights = np.array([1.0, 4.0, 1.0])

In [425]:
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [426]:
prediction = predict_outcome(normalized_feature_matrix, initial_weights)

In [427]:
ro_1 = np.sum(normalized_feature_matrix[:, 1] * 
              (output - prediction + initial_weights[1] * normalized_feature_matrix[:, 1]))
print ("ro_1: %.0f, 2*ro_1: %.0f" % ( ro_1, 2 * ro_1))

ro_1: 87939471, 2*ro_1: 175878942


In [428]:
ro_2 = np.sum(normalized_feature_matrix[:, 2] * 
              (output - prediction + initial_weights[2] * normalized_feature_matrix[:, 2]))
print ("ro_2: %.0f, 2*ro_2: %.0f" % ( ro_2, 2 * ro_2))

ro_2: 80966699, 2*ro_2: 161933397


In [429]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


### Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.
Return weights

The function should accept the following parameters:

Feature matrix
Output array
Initial weights
L1 penalty
Tolerance
e.g. in Python:

In [387]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights.copy()
    should_continue = True
    iter_cnt = 0
    
    while should_continue:
        max_change = 0.0
        for i in range(len(weights)):
            old_weight_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, 
                                                       weights, l1_penalty)
            max_change = max(max_change, abs(weights[i] - old_weight_i))
        
        iter_cnt += 1
        should_continue = max_change >= tolerance
        print ("iter # %d, max_change = %f" % (iter_cnt, max_change))

    return weights

Let us now go back to the simple model with 2 features: `sqft_living` and `bedrooms`. Using `get_numpy_data` (or equivalent), extract the feature matrix and the output array from from the house dataframe. Then normalize the feature matrix using `normalized_features()` function.

Using the following parameters, learn the weights on the sales dataset.

- Initial weights = all zeros
- L1 penalty = 1e7
- Tolerance = 1.0

In [388]:
feature_matrix

array([[1.00e+00, 1.18e+03, 3.00e+00],
       [1.00e+00, 2.57e+03, 3.00e+00],
       [1.00e+00, 7.70e+02, 2.00e+00],
       ...,
       [1.00e+00, 1.02e+03, 2.00e+00],
       [1.00e+00, 1.60e+03, 3.00e+00],
       [1.00e+00, 1.02e+03, 2.00e+00]])

In [389]:
normalized_feature_matrix, _ = normalize_features(feature_matrix)

In [390]:
initial_weights = np.zeros(normalized_feature_matrix.shape[1])
l1_penalty = 1e7
tolerance = 1.0

In [391]:
weight_learned_by_lasso = lasso_cyclical_coordinate_descent(normalized_feature_matrix,
                                                            output,
                                                            initial_weights,
                                                            l1_penalty,
                                                            tolerance
                                                           )

iter # 1, max_change = 79400304.637645
iter # 2, max_change = 9138168.376428
iter # 3, max_change = 8194809.518383
iter # 4, max_change = 6598905.081920
iter # 5, max_change = 5522173.230820
iter # 6, max_change = 4621129.840878
iter # 7, max_change = 3867108.131826
iter # 8, max_change = 3236118.832011
iter # 9, max_change = 2708086.957463
iter # 10, max_change = 2266213.124388
iter # 11, max_change = 1896439.075191
iter # 12, max_change = 1587000.413689
iter # 13, max_change = 1328052.319738
iter # 14, max_change = 1111356.335353
iter # 15, max_change = 930018.257393
iter # 16, max_change = 778268.797837
iter # 17, max_change = 651280.033346
iter # 18, max_change = 545011.804423
iter # 19, max_change = 456083.177361
iter # 20, max_change = 381664.879519
iter # 21, max_change = 319389.285747
iter # 22, max_change = 267275.092165
iter # 23, max_change = 223664.280801
iter # 24, max_change = 187169.369584
iter # 25, max_change = 156629.269480
iter # 26, max_change = 131072.344328
iter #

In [392]:
weight_learned_by_lasso

array([21624997.9595191 , 63157247.20788956,        0.        ])

### Quiz Question: What is the RSS of the learned model on the normalized dataset?





In [393]:
prediction = predict_outcome(normalized_feature_matrix, weight_learned_by_lasso)
errors = prediction - output

In [394]:
rss_normalized_dataset = np.sum(errors * errors)
rss_normalized_dataset

1630492476715386.5

### 16. Quiz Question: Which features had weight zero at convergence?

'bedrooms'

## Evaluating LASSO fit with more features

In [395]:
train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

Create a normalized feature matrix from the TRAINING data with the following set of features.

[bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated]
- Make sure you store the norms for the normalization, since we’ll use them later.

In [396]:
all_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
                'floors', 'waterfront', 'view', 'condition', 'grade',
                'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [397]:
(training_feature_matrix, training_output) = get_numpy_data(train, all_features, 'price')

In [398]:
normalized_training_feature_matrix, features_norms = normalize_features(training_feature_matrix)

In [399]:
normalized_training_feature_matrix

array([[0.00758447, 0.00652117, 0.0033687 , ..., 0.        , 0.00752148,
        0.        ],
       [0.00758447, 0.00652117, 0.00757957, ..., 0.0057043 , 0.0075061 ,
        0.03707954],
       [0.00758447, 0.00434745, 0.0033687 , ..., 0.        , 0.00743684,
        0.        ],
       ...,
       [0.00758447, 0.00652117, 0.00842175, ..., 0.        , 0.00772924,
        0.        ],
       [0.00758447, 0.00652117, 0.00842175, ..., 0.        , 0.00771   ,
        0.        ],
       [0.00758447, 0.00434745, 0.00252652, ..., 0.        , 0.00772539,
        0.        ]])

 First, learn the weights with `l1_penalty=1e7`, on the training data. 
 Initialize weights to all zeros, and set the `tolerance=1`. 
 Call resulting weights `weights1e7`, you will need them later.

In [400]:
l1_penalty = 1e7
initial_weights = np.zeros(1 + len(all_features))
tolerance = 1.0

In [401]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_training_feature_matrix, 
                                                        training_output, 
                                                     initial_weights,
                                                     l1_penalty,
                                                     tolerance
                                                    )

weights1e7

iter # 1, max_change = 71114625.714887
iter # 2, max_change = 5024356.370547
iter # 3, max_change = 5012324.915982
iter # 4, max_change = 5000676.422621
iter # 5, max_change = 4991243.174673
iter # 6, max_change = 4984361.344820
iter # 7, max_change = 4979504.930990
iter # 8, max_change = 4973806.566812
iter # 9, max_change = 2879223.018989
iter # 10, max_change = 2706150.775285
iter # 11, max_change = 2532295.658295
iter # 12, max_change = 2368316.587067
iter # 13, max_change = 2213124.359407
iter # 14, max_change = 1552443.881222
iter # 15, max_change = 507838.022651
iter # 16, max_change = 438814.364292
iter # 17, max_change = 371479.617658
iter # 18, max_change = 313415.101470
iter # 19, max_change = 264244.828686
iter # 20, max_change = 222753.857891
iter # 21, max_change = 187770.664855
iter # 22, max_change = 158280.091291
iter # 23, max_change = 133420.899865
iter # 24, max_change = 112465.986828
iter # 25, max_change = 94802.212214
iter # 26, max_change = 79912.686407
iter # 2

array([24429600.23440313,        0.        ,        0.        ,
       48389174.77154895,        0.        ,        0.        ,
        3317511.21492166,  7329961.81171426,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

### Quiz Question: What features had non-zero weight in this case?

In [402]:
intercept_with_features = ['constant'] + all_features
nonzeros_features = [intercept_with_features[i] for i in range(len(intercept_with_features))
                     if (weights1e7[i] != 0)]
nonzeros_features

['constant', 'sqft_living', 'waterfront', 'view']

Next, learn the weights with `l1_penalty=1e8`, on the training data. 
Initialize weights to all zeros, and set the `tolerance=1`.
Call resulting weights `weights1e8`, you will need them later.



In [403]:
l1_penalty= 1e8
tolerance = 1.
initial_weights=np.zeros(1 + len(all_features))

weights1e8 = lasso_cyclical_coordinate_descent(
    normalized_training_feature_matrix,
    training_output,
    initial_weights,
    l1_penalty,
    tolerance
)

iter # 1, max_change = 71114625.714887
iter # 2, max_change = 0.000000


In [404]:
weights1e8

array([71114625.71488702,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

### Quiz Question: What features had non-zero weight in this case?

In [405]:
# Constant

Finally, learn the weights with `l1_penalty=1e4`, on the training data. 
Initialize weights to all zeros, and set the `tolerance=5e5`. 
Call resulting weights `weights1e4`, you will need them later. 
(This case will take quite a bit longer to converge than the others above.)


In [406]:
l1_penalty= 1e4
tolerance = 1e5
initial_weights=np.zeros(1 + len(all_features))

weights1e4 = lasso_cyclical_coordinate_descent(
    normalized_training_feature_matrix,
    training_output,
    initial_weights,
    l1_penalty,
    tolerance
)


iter # 1, max_change = 71114625.714887
iter # 2, max_change = 8441276.045842
iter # 3, max_change = 6789873.036988
iter # 4, max_change = 5538914.174014
iter # 5, max_change = 4527797.915308
iter # 6, max_change = 3700363.669842
iter # 7, max_change = 3022494.096842
iter # 8, max_change = 2468071.048076
iter # 9, max_change = 2015787.324400
iter # 10, max_change = 1647983.509669
iter # 11, max_change = 1389754.619568
iter # 12, max_change = 1334787.182747
iter # 13, max_change = 1293147.731587
iter # 14, max_change = 1261718.382102
iter # 15, max_change = 1238001.623368
iter # 16, max_change = 1220007.877715
iter # 17, max_change = 1206162.336051
iter # 18, max_change = 1194973.111592
iter # 19, max_change = 1185736.490952
iter # 20, max_change = 1177721.687410
iter # 21, max_change = 1170366.252274
iter # 22, max_change = 1163250.460227
iter # 23, max_change = 1156068.165101
iter # 24, max_change = 1148603.014891
iter # 25, max_change = 1140709.161491
iter # 26, max_change = 1132295.6

iter # 214, max_change = 317075.967540
iter # 215, max_change = 316067.114912
iter # 216, max_change = 315062.433221
iter # 217, max_change = 314061.910422
iter # 218, max_change = 313065.534102
iter # 219, max_change = 312073.291509
iter # 220, max_change = 311085.169574
iter # 221, max_change = 310101.154936
iter # 222, max_change = 309121.233962
iter # 223, max_change = 308145.392769
iter # 224, max_change = 307173.617245
iter # 225, max_change = 306205.893065
iter # 226, max_change = 305242.205710
iter # 227, max_change = 304282.540485
iter # 228, max_change = 303326.882533
iter # 229, max_change = 302375.216852
iter # 230, max_change = 301427.528309
iter # 231, max_change = 300483.801652
iter # 232, max_change = 299544.021522
iter # 233, max_change = 298608.172471
iter # 234, max_change = 297676.238966
iter # 235, max_change = 296748.205404
iter # 236, max_change = 295824.056120
iter # 237, max_change = 294903.775400
iter # 238, max_change = 293987.347485
iter # 239, max_change = 

iter # 431, max_change = 172761.467864
iter # 432, max_change = 172352.496190
iter # 433, max_change = 171945.210101
iter # 434, max_change = 171539.602513
iter # 435, max_change = 171135.666372
iter # 436, max_change = 170733.394655
iter # 437, max_change = 170332.780366
iter # 438, max_change = 169933.816540
iter # 439, max_change = 169536.496241
iter # 440, max_change = 169140.812561
iter # 441, max_change = 168746.758622
iter # 442, max_change = 168354.327574
iter # 443, max_change = 167963.512596
iter # 444, max_change = 167574.306895
iter # 445, max_change = 167186.703707
iter # 446, max_change = 166800.696297
iter # 447, max_change = 166416.277956
iter # 448, max_change = 166033.442005
iter # 449, max_change = 165652.181792
iter # 450, max_change = 165272.490694
iter # 451, max_change = 164894.362114
iter # 452, max_change = 164517.789485
iter # 453, max_change = 164142.766264
iter # 454, max_change = 163769.285939
iter # 455, max_change = 163397.342024
iter # 456, max_change = 

iter # 648, max_change = 113475.262406
iter # 649, max_change = 113307.662792
iter # 650, max_change = 113140.793032
iter # 651, max_change = 112974.644930
iter # 652, max_change = 112809.210702
iter # 653, max_change = 112644.482940
iter # 654, max_change = 112480.454585
iter # 655, max_change = 112317.118905
iter # 656, max_change = 112154.469472
iter # 657, max_change = 111992.500139
iter # 658, max_change = 111831.205020
iter # 659, max_change = 111670.578478
iter # 660, max_change = 111510.615100
iter # 661, max_change = 111351.309691
iter # 662, max_change = 111192.657252
iter # 663, max_change = 111034.652974
iter # 664, max_change = 110877.292220
iter # 665, max_change = 110720.570520
iter # 666, max_change = 110564.483556
iter # 667, max_change = 110409.027152
iter # 668, max_change = 110254.197272
iter # 669, max_change = 110099.990002
iter # 670, max_change = 109946.401551
iter # 671, max_change = 109793.428238
iter # 672, max_change = 109641.066487
iter # 673, max_change = 

In [407]:
weights1e4

array([ 1.28874098e+08, -1.80658455e+07,  1.61398374e+05,  1.87035801e+08,
       -1.62304457e+06,  2.35166532e+05,  6.75281534e+06,  5.54075037e+06,
        2.07452320e+07,  1.08623137e+08, -1.12704447e+08, -2.89135400e+07,
       -2.23652499e+08,  3.03702632e+06])

### 4. Quiz Question: What features had non-zero weight in this case?

In [408]:
intercept_with_features = ['constant'] + all_features
nonzeros_features = [intercept_with_features[i] for i in range(len(intercept_with_features))
                     if (weights1e4[i] != 0)]
nonzeros_features

['constant',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

## Rescaling learned weights

Create a normalized version of each of the weights learned above. (‘weights1e4’, ‘weights1e7’, ‘weights1e8’). 
To check your results, if you call ‘normalized_weights1e7’ the normalized version of ‘weights1e7’, then
```python
print normalized_weights1e7[3]
```
should print `161.31745624837794.`

In [409]:
normalized_weights1e4, normalized_weights1e7, normalized_weights1e8 = weights1e4/features_norms, weights1e7/features_norms, weights1e8/features_norms

In [410]:
normalized_weights1e7[3]

161.3174576461176

### compute rss on test set

In [411]:
test_feature_matrix, test_output = get_numpy_data(test, all_features, 'price')

In [412]:
predicted_test_1e4 = predict_outcome(test_feature_matrix, normalized_weights1e4)
errors_1e4 = predicted_test_1e4 - test_output
rss_test_1e4 = np.sum(errors_1e4 * errors_1e4)
rss_test_1e4

205525047738808.38

In [413]:
predicted_test_1e7 = predict_outcome(test_feature_matrix, normalized_weights1e7)
errors_1e7 = predicted_test_1e7 - test_output
rss_test_1e7 = np.sum(errors_1e7 * errors_1e7)
rss_test_1e7

275962075920366.8

In [414]:
predicted_test_1e8 = predict_outcome(test_feature_matrix, normalized_weights1e8)
errors_1e8 = predicted_test_1e8 - test_output
rss_test_1e8 = np.sum(errors_1e8 * errors_1e8)
rss_test_1e8

537166151497322.75

### Quiz Question: Which model performed best on the test data?

In [415]:
min(rss_test_1e4, rss_test_1e7, rss_test_1e8)
# rss_test_1e4

205525047738808.38